In [1]:
from pyspark.sql import SparkSession
from IPython.display import display

# Apache Spark

In [ ]:
spark = SparkSession.builder.appName("Intro_spark").getOrCreate() \
        #.config("spark.driver.bindAddress","localhost") \
        #.config("spark.ui.port","4050") \
        

# check if Adaptive Query Execution (AQE) is enabled
print(spark.conf.get("spark.sql.adaptive.enabled"))

df = spark.read.parquet("yellow_tripdata_2023-05.parquet")
# print(df.rdd.getNumPartitions())
# df.count()
# df.explain()
display(df.toPandas())

your 131072x1 screen size is bogus. expect trouble
25/03/16 09:20:11 WARN Utils: Your hostname, Amaneul-PC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/16 09:20:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/16 09:20:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


true


Py4JJavaError: An error occurred while calling o29.collectToPython.
: java.lang.OutOfMemoryError: GC overhead limit exceeded
	at org.apache.spark.sql.execution.SparkPlan$$anon$1._next(SparkPlan.scala:417)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.getNext(SparkPlan.scala:426)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.getNext(SparkPlan.scala:412)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at org.apache.spark.util.NextIterator.foreach(NextIterator.scala:21)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1(SparkPlan.scala:449)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1$adapted(SparkPlan.scala:448)
	at org.apache.spark.sql.execution.SparkPlan$$Lambda$2944/2046730514.apply(Unknown Source)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.co

In [6]:
print(f"{spark.sparkContext.defaultParallelism} cores available")

24 cores available


## Spark Basics

### User Defined Functions (UDFs)

#### Python UDF

Generally the slowest kind of UDF as it applies the function to one row at a time.

### Pandas UDF

Faster than Python UDFs as they apply the function to *batches* of rows (block retrieval) at a time and also because it leverages Apache Arrow for faster serialization (converting the transformed rows back into Java objects to pass back to the JVM).

## Spark Architecture 

In Spark computations are broken into to two rough categories: transformations and actions.

### Transformations

An important thing to note about transformations in Spark is that, generally, they are *lazy* operations; meaning they aren't accualy performed/computed until the resulting dataframe/result set is needed. Generally this means the transformation isn't applied until an *action* is specified. 

There are two types of transformations: narrow and wide transformations. The difference between the two is that *wide* transformations induce what is called a *shuffle* (also called an *exchange*). A shuffle is when a *repartitioning* of the dataset is required. Spark basically works by partitioning the dataset into subsets and having each worker perform the specified computation on a partition of the data. It's easy to think of transformations that would require a shuffle such as the `sort()` operation. Clearly this will require an exchange/shuffle since one needs to compare rows from different partitions to perform a sort. And example of a narrow transformation is the `filter()` transformation. Clearly there is no need to compare rows across partitions to accomplish this operation since we just need to check if a row fulfills the criteria in the filter clause to compute the result set.

Shuffle vs memory partitions

Heirarchy 
    Driver 
        ├── Workers
                ├── Executors
                           ├── Cores

## Structured Streaming

## Practice questions

These are practice exam questions from the March 2025 practice exam provided by Databricks for the 'Databricks Certified Associate Developer for Apache Spark 3.0 - Python' certification exam. The pdf of the exam is available in the `pdfs/` folder of this section in the JupyterBook.

### Question 1